In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, ElasticNetCV,RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
oppScrData = pd.read_excel(r'/Users/sanyamkakkar/Desktop/Spring/Ml760/CS760-Project/OppScrData.xlsx')

In [3]:
def clean_ct_data(oppScrData):
    # Delete rows with empty values
    cols = ["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]']
    ct_data= oppScrData[cols]
    n = ct_data.shape[0]
    preprocessed_ct_data = pd.DataFrame( columns=cols)
    for i in range(n):
        row = ct_data.loc[i]
        ignore = False
        for k,j in enumerate(row):
          if pd.isna(j) or j == ' ': 
            if k == 11:              
              ignore = False
            else:
              ignore = True
              break
        if not ignore:
          preprocessed_ct_data.loc[i]= row
    return preprocessed_ct_data

In [4]:
sc = StandardScaler()
ct_data = clean_ct_data(oppScrData=oppScrData)

In [5]:
non_null_death_rows = ct_data[~ct_data['DEATH [d from CT]'].isnull()]
print(non_null_death_rows.shape)
print(non_null_death_rows.columns)

(493, 12)
Index(['L1_HU_BMD', 'TAT Area (cm2)',
       'Total Body                Area EA (cm2)', 'VAT Area (cm2)',
       'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]'],
      dtype='object')


In [6]:
# Convert to years
non_null_death_rows['DEATH [d from CT]'] /= 365

X = non_null_death_rows.iloc[:, 0:11]
y = non_null_death_rows.iloc[:, 11]
X = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

null_death_rows=ct_data[ct_data['DEATH [d from CT]'].isnull()]
X2 = null_death_rows.iloc[:, 0:11]
X2 = pd.DataFrame(sc.fit_transform(X2), columns = X2.columns)

<ipython-input-6-04a09313e2a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_null_death_rows['DEATH [d from CT]'] /= 365


In [7]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
def testWithModel(model, X_train, X_test, y_train, y_test):
  clf = model
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)

  print(f'MAE = {mean_absolute_error(y_test, y_pred)}') 
  print(f'MAPE = {mean_absolute_percentage_error(y_test, y_pred)}')
  print(f'RMSE = {mean_squared_error(y_test, y_pred)**0.5}')

  # plot_results(y_test, y_pred)

  return y_pred

In [9]:
RFparams2={'n_estimators':[50],
          'min_samples_split':[16],
          'min_samples_leaf': [4],
           'max_features': [6],
           'max_depth': [20],
           'bootstrap': [True]}

rf = RandomForestRegressor(random_state = 21)
clf_random_forest = GridSearchCV(rf, RFparams2, refit=True)
testWithModel(clf_random_forest ,X_train, X_test, y_train, y_test)

MAE = 2.7602421790301745
MAPE = 4.059821074779301
RMSE = 3.3082769109535644


array([7.12646891, 4.52820236, 7.43140678, 6.26865563, 6.6520216 ,
       4.29461668, 7.26303157, 6.05189659, 8.5200816 , 5.16621571,
       6.58877644, 6.71604273, 6.82889658, 5.76299639, 5.64790579,
       5.7620733 , 6.5630654 , 7.11510729, 8.05580818, 6.87189164,
       7.58776774, 5.03678048, 6.30506429, 7.53166628, 6.93814732,
       7.85002022, 6.69365883, 7.19493297, 7.8117013 , 6.13612377,
       7.28411387, 6.70982052, 7.34311763, 5.78209986, 6.431361  ,
       6.84182256, 6.50171466, 7.82215326, 5.64445545, 7.6331574 ,
       7.69445925, 7.46985147, 7.25116408, 7.32821106, 7.14887591,
       7.85434355, 4.60922002, 6.39408042, 7.38741957, 5.41984801,
       7.99944692, 6.0574601 , 5.74616106, 5.9350917 , 7.25463176,
       6.04227286, 5.48574893, 5.20061761, 5.62240827, 5.42631831,
       5.16864281, 5.98022717, 7.26111663, 5.60770385, 7.16404675,
       6.21400837, 6.00117844, 5.44585726, 6.34259672, 7.89652856,
       8.93778496, 6.4099171 , 8.18246326, 5.66963888, 5.07408

In [10]:
testWithModel(Lasso(alpha=0.5), X_train, X_test, y_train, y_test)

MAE = 2.777563760826399
MAPE = 3.784858071420441
RMSE = 3.311534916910759


array([6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681385,
       6.31681385, 6.31681385, 6.31681385, 6.31681385, 6.31681

In [11]:
testWithModel(RidgeCV(),X_train, X_test, y_train, y_test)

MAE = 2.7476411376271224
MAPE = 4.0386224203656855
RMSE = 3.2960287236612618


array([7.10713272, 4.67366429, 6.54988611, 6.65723777, 3.10200041,
       5.27965676, 6.88324595, 6.61032737, 5.89276583, 5.10867565,
       6.94022439, 6.52836871, 7.15579173, 7.05912637, 6.58991331,
       7.36294417, 6.36240616, 7.25705289, 6.6361233 , 5.7611579 ,
       8.05283963, 5.82483796, 6.73644857, 6.65118939, 7.07799335,
       7.01411571, 7.22116582, 7.12450664, 6.11689469, 5.43038526,
       5.99527289, 6.9864053 , 6.10998109, 7.22470093, 6.5990619 ,
       6.89951506, 6.94161165, 6.37445423, 3.97299622, 5.34106392,
       6.88452528, 6.89229669, 6.27785702, 6.10548695, 7.44006661,
       5.29414539, 4.61104048, 6.89413221, 6.16553986, 6.7981388 ,
       6.49210272, 6.8138075 , 6.93899161, 5.50534976, 6.03798157,
       6.737974  , 6.11616064, 6.5880536 , 6.24834908, 4.85375897,
       5.16227773, 6.17109809, 7.60538386, 6.50447718, 5.6915078 ,
       6.3247921 , 6.30913187, 5.88658769, 7.05389792, 5.13462111,
       6.09782396, 4.96536108, 6.25748324, 5.68506391, 6.32384

In [13]:
testWithModel(KNeighborsRegressor(),X_train, X_test, y_train, y_test)

MAE = 2.719313684793137
MAPE = 4.192761817399197
RMSE = 3.3737524283780953


array([ 9.67452055,  4.37643836,  9.04273973,  5.87671233,  5.7490411 ,
        6.2       ,  5.60767123,  4.38794521,  9.54027397,  5.70684932,
        8.05260274,  6.22739726,  7.75123288,  7.08821918,  7.63013699,
        3.42191781,  7.5309589 ,  6.83506849,  6.88986301,  8.87013699,
        7.96164384,  8.05205479,  4.63561644,  9.38520548,  4.78684932,
        7.67780822,  5.16273973,  6.94082192,  9.76493151,  5.30575342,
        6.29753425,  7.5769863 ,  7.67945205,  5.32712329,  7.15013699,
        6.4       ,  6.91178082,  6.8630137 ,  6.45808219,  8.25643836,
        7.19561644,  7.54520548,  7.33205479,  9.08657534,  5.0169863 ,
        4.51726027,  6.45753425,  8.46465753,  8.65589041,  8.8969863 ,
        6.44712329,  4.63232877,  5.58027397,  7.26465753,  6.94684932,
        6.95616438,  5.1939726 ,  6.05534247,  4.53534247,  4.92876712,
        6.31287671,  6.96712329,  9.07123288,  5.23726027,  4.79123288,
        7.20109589,  3.90191781,  7.11342466, 10.2969863 ,  7.09

In [14]:
testWithModel(DecisionTreeRegressor(criterion="absolute_error"),X_train, X_test, y_train, y_test)

MAE = 4.020893870208939
MAPE = 4.855472914675468
RMSE = 4.8015805775448435


array([ 3.28767123,  2.89863014,  5.39726027,  5.98082192, 11.76712329,
        5.38356164,  8.85753425,  0.8630137 ,  8.54246575,  4.19452055,
        9.38082192,  0.71506849,  9.50410959,  9.04931507,  0.8630137 ,
        0.59452055, 11.10136986,  1.09863014, 13.13424658,  7.5260274 ,
        7.47671233, 11.95616438,  0.5369863 ,  9.82465753,  0.85753425,
        8.83013699,  0.02465753, 10.2       , 11.40547945,  0.20273973,
        1.42465753, 12.16712329,  5.84383562, 12.97534247, 12.09315068,
        0.71506849,  7.47671233,  8.02739726,  4.55342466,  6.74246575,
        8.68493151,  9.41369863,  7.42191781,  5.52054795,  9.38630137,
        1.78356164,  9.20273973, 10.51232877,  9.41369863,  4.91506849,
       12.63013699,  2.0739726 ,  4.19452055,  9.30136986,  6.70684932,
        6.01369863,  0.99178082,  4.25753425,  9.86027397,  2.73424658,
        2.51232877,  7.30410959, 13.19178082, 13.89041096,  9.50410959,
        7.79452055, 13.35616438,  5.38356164,  7.16164384,  7.03

In [15]:
testWithModel(ElasticNetCV(random_state=0), X_train, X_test, y_train, y_test)

MAE = 2.748094198258288
MAPE = 4.0262614026053205
RMSE = 3.295113916027013


array([7.07450256, 4.72022492, 6.53070663, 6.6556009 , 3.20459659,
       5.30675804, 6.85076739, 6.63252683, 5.9008198 , 5.13464215,
       6.92721964, 6.51893789, 7.09858763, 7.03256087, 6.56495874,
       7.31002423, 6.35539431, 7.22527951, 6.63670077, 5.78001699,
       7.97160746, 5.85653407, 6.71855205, 6.63444385, 7.05298444,
       6.99078205, 7.19438953, 7.09189238, 6.11679198, 5.46803537,
       6.00913978, 6.95933149, 6.1155397 , 7.20680787, 6.57199778,
       6.87219627, 6.94311471, 6.36492849, 4.01487185, 5.37419474,
       6.86763033, 6.87886797, 6.27412829, 6.12444763, 7.43435871,
       5.31866377, 4.65677513, 6.87332767, 6.16200981, 6.79268654,
       6.48671454, 6.7927389 , 6.89694862, 5.53547756, 6.05173816,
       6.72252651, 6.18009862, 6.57562573, 6.24860817, 4.8898252 ,
       5.15857035, 6.17585083, 7.54113394, 6.53502196, 5.72744133,
       6.30324053, 6.3235687 , 5.89374056, 7.01929817, 5.16963955,
       6.09007556, 5.01014813, 6.25860082, 5.70595012, 6.29798

In [16]:
testWithModel(SVR(C=300, epsilon=0.8),X_train, X_test, y_train, y_test)

MAE = 3.6856102596448257
MAPE = 4.43877691849709
RMSE = 4.619371975351302


array([ 6.36626015,  1.02622363,  9.0837484 ,  1.24905069,  4.54120106,
        3.29927264,  7.06854901,  1.5340205 ,  6.48494171,  5.12615895,
       10.48046634,  4.29371794,  8.1229    ,  6.92281516,  3.33314288,
        4.84237351,  1.38846946,  9.01510055,  4.35414608,  8.68010435,
        4.46947759,  5.52557108,  4.47351019,  8.35877231,  4.61942405,
        6.68930224, 10.45131078,  8.15085113,  6.80308361,  5.84649968,
        0.49200731,  6.48516348,  7.96116055,  4.52661451,  3.87681909,
        5.98400698, 13.08853089,  7.18300142,  4.81242731,  8.58769603,
        9.43727377,  5.73332703,  8.76665783, 15.04016235,  7.2016792 ,
        7.81561256,  4.00588244,  6.43665482,  3.08703067, 15.48227537,
        7.0622163 ,  5.98948944, -0.09848996,  6.43054265,  8.7112775 ,
        7.05026656, 13.28216064,  4.16948151,  6.26907261,  3.33016224,
        3.61463292,  4.64886254, 10.51319407,  1.33670849, 10.01793234,
        8.17821416,  3.80725249,  7.38055846, 10.89358774,  4.69

In [17]:
null_death_rows['DEATH [d from CT]'] = clf_random_forest.predict(X2)
df = pd.concat([non_null_death_rows,null_death_rows])
y = df.iloc[:,11]
X = df.iloc[:, 0:11]

<ipython-input-17-b107bf54d458>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_death_rows['DEATH [d from CT]'] = clf_random_forest.predict(X2)


In [18]:
print(X.shape)

(8878, 11)


In [19]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)

In [20]:
knn = KNeighborsRegressor(n_neighbors=9)
param_grid2 = dict(weights = ["uniform", "distance"])
knn_pred = testWithModel(GridSearchCV(knn, param_grid2),X_train, X_test, y_train, y_test)

MAE = 0.6598922564075452
MAPE = 0.1307413503893314
RMSE = 1.0569580649579173


In [21]:
svr_pred = testWithModel(SVR(C=300, epsilon=0.8),X_train, X_test, y_train, y_test)

MAE = 0.663446592517845
MAPE = 0.12837231099786717
RMSE = 1.0539182120521493


In [22]:
lasso_pred = testWithModel(Lasso(alpha=0.1), X_train, X_test, y_train, y_test)

MAE = 0.8392944341607153
MAPE = 0.15378502700928734
RMSE = 1.179094969387008


In [23]:
knn_pred = testWithModel(KNeighborsRegressor(),X_train, X_test, y_train, y_test)

MAE = 0.682850758434054
MAPE = 0.1339451671387361
RMSE = 1.0875734813493776


In [24]:
elastic_net_pred = testWithModel(ElasticNetCV(random_state=0), X_train, X_test, y_train, y_test)

MAE = 0.8453674892573712
MAPE = 0.1553148980159573
RMSE = 1.1870191048664442


In [25]:
dtree_pred = testWithModel(DecisionTreeRegressor(criterion="absolute_error"),X_train, X_test, y_train, y_test)

MAE = 0.7442915789775519
MAPE = 0.1378816736563948
RMSE = 1.3849886619549137


In [26]:
array_preds = np.array([ dtree_pred, elastic_net_pred, knn_pred, lasso_pred,dtree_pred])
y_ens = np.mean((array_preds), axis=0 )

In [27]:
print(mean_absolute_error(y_ens, y_test))

0.6447868309830067


In [28]:
print(array_preds.shape)

(5, 1776)


In [29]:
import random
y_ens=[]

for i,y in enumerate(y_test):

  j = random.randint(0, 4)
  y_ens.append( array_preds[j][i])

In [30]:
len(y_ens)

1776

In [31]:
print(mean_absolute_error(y_ens, y_test))

0.7774032951412713


In [34]:
array_preds

array([[7.82075502, 7.1578268 , 6.70649182, ..., 6.99684322, 5.11695048,
        8.48470691],
       [6.81910098, 7.50140208, 7.00772099, ..., 6.7066949 , 6.09621763,
        6.72406719],
       [7.90883059, 6.64446685, 7.04434974, ..., 6.7283526 , 4.85708702,
        8.40040148],
       [6.88671246, 7.5201356 , 6.97558499, ..., 6.69259602, 6.11212507,
        6.76183802],
       [7.82075502, 7.1578268 , 6.70649182, ..., 6.99684322, 5.11695048,
        8.48470691]])

In [35]:
b = array_preds.T

In [37]:
b

X_trainens, X_testens, y_trainens, y_testens  = train_test_split(b, y_test, test_size=0.2, random_state=0)

In [38]:
from sklearn.linear_model import LinearRegression


reg = LinearRegression().fit(X_trainens, y_trainens)

reg.score(X_testens, y_testens)

0.5509104237435773

In [41]:
reg.coef_

array([ 0.10756112, -1.33566529,  0.56674049,  1.77740378,  0.10756112])

In [43]:
reg.intercept_

-1.473247958163018

In [44]:
y_ens=reg.predict(b)

In [45]:
print(mean_absolute_error(y_ens, y_test))

0.6030037053409997


In [49]:
print(f'MAPE = {mean_absolute_percentage_error(y_test, y_ens)}')

MAPE = 0.11610194686413265
